In [1]:
import pandas as pd
import numpy as np
import re
from pandas import ExcelWriter
from pandas import ExcelFile
from analysis import DataHandler, get_table
from sqlite3 import DatabaseError, OperationalError
import matplotlib.pyplot as plt
from call import folder, read_scenarios

%load_ext autoreload
%autoreload 2

In [2]:
pd.options.mode.chained_assignment = None

scenario_file = 'scenarios_SENTINEL_CS_EU'

scenarios = read_scenarios(scenario_file + '.yaml')

dh = DataHandler(scenarios)

In [3]:
index = ['Model','Scenario','Region','Updated code','Unit','Year']

maps = {
    'technology': {
        'all_CCS' :  '|CCS',
        'all_coal':  '|Coal',
        'all_coal_CCS': '|Coal|CCS',
        'CCGT':      '|Gases|Fossil|Natural gas',
        'CCGT_H2':   '|Gases|Hydrogen',
        'OCGT':      '|Gases|Fossil|Natural gas',
        'OCGT_H2':   '|Gases|Hydrogen',
        'PHS':       '|Electricity Storage|Medium Duration',
        'PtHydrogen':'|Electricity',
        'batr':      '|Electricity Storage|Short Duration',
        'sto':       '|Electricity Storage',
        'bio':       '|Solar bio and waste',
        'coal':      '|Coal|Hard Coal',
        'hydr':      '|Hydro|dam',
        'hyd':       '|Hydro',
        'lign':      '|Coal|Brown Coal|Lignite',
        'lign_CCS':  '|Coal|Brown Coal|Lignite|CCS',
        'coal_CCS':  '|Coal|Hard Coal|CCS',
        'CCGT_CCS':  '|Gases|Fossil|Natural gas|CCS',
        'nucl':      '|Nuclear',
        'ror':       '|Hydro|river',
        'shed':      'flex_est_und',
        'solar':     '|Solar',
        'wind':      '|Wind',
        'wiof':      '|Wind|Offshore',
        'wion':      '|Wind|Onshore',
        'ntc':       '|Interconnect Importing Capacity',
        'all':       '',
    },
    'scenario_name': {
        '2016':      'Reference_year',
        '2030_EU_CT':'Current trends',
        '2050_EU_CT':'Current trends',
        '2030_EU_CN':'Climate neutrality',
        '2050_EU_CN':'Climate neutrality',
        '2040_EU_EN':'Early neutrality'
    },
    'categories': {
        'Electricity': ['CCGT','CCGT_H2','OCGT','OCGT_H2','bio','coal','hydr','lign','lign_CCS','coal_CCS','CCGT_CCS','nucl','ror','solar','wiof','wion'],
        'Flexibility': ['batr','PHS','ntc'],
        'Hydrogen': ['PtHydrogen'],
        'Heat': ['lign','lign_CCS','coal','coal_CCS','CCGT','OCGT','CCGT_CCS','CCGT_H2','OCGT_H2']
    },
    'categories_agg_techs': {
        'Electricity': ['all','all_CCS','all_coal','all_coal_CCS','hyd','wind'],
        'Flexibility': ['sto'],
        'Hydrogen': [],
        'Heat': ['all','all_CCS','all_coal','all_coal_CCS'],
    },
    'aggregated_techs': {
        'all': ['CCGT','CCGT_H2','OCGT','OCGT_H2','bio','coal','hydr','lign','lign_CCS','coal_CCS','CCGT_CCS','nucl','ror','shed','solar','wiof','wion'],
        'all_CCS': ['CCGT_CCS','coal_CCS','lign_CCS'],
        'all_coal': ['coal','lign'],
        'all_coal_CCS': ['lign_CCS','coal_CCS'],
        'sto': ['batr','PHS'],
        'hyd': ['ror','hydr'],
        'wind': ['wiof','wion']
    },
    'unit_per_category': {
        'Installed capacity': 'GW',
        'Generation|Yearly': 'GWh',
        'Generation|Summer peak': 'GWh',
        'Generation|Winter peak': 'GWh',
        'Generation|Percentile 25': 'GWh',
        'Generation|Percentile 50': 'GWh',
        'Fuel consumption': 'TJ/year',
        'Efficiency': 'dimensionless',
        'Emissions|Kyoto Gases|Fossil|CO2': 'Mt CO2/year',
        'Investments': 'Time frame,  billion Euros (PPP, 2015)'
    }
}

to_scenarioID = {str(val['name']):key for key, val in scenarios.items()}

def change_tec_lvl_name_to_alltec(names):
    '''
    Function changes level name of technology level (for example tec_supply) to alltec
    '''
    new_names = []
    for i in range(len(names)):
        m = re.search('.*tec.*',str(names[i]))
        if m != None:
            new_names.append('alltec')
        else:
            new_names.append(names[i])
    return new_names

def replace(val, d:dict, prepend:str='', check:bool=True):
    try:
        return prepend+str(d[val])
    except KeyError:
        if check:
            raise
        else:
            return val

def map_level(idx, dct, level=0):
    new_idx = idx.set_levels([[dct.get(item, item) for item in names] if i==level else names
                      for i, names in enumerate(idx.levels)])
    return new_idx

def create_expanded_tech_MultiIndex(mI,category):
    i = [mI.get_level_values(name).unique() for name in mI.names if name not in ['alltec','scenario']]
    i_alltec = mI.names.index('alltec')
    i.insert(i_alltec,maps['categories'][category])
    i_scenario = mI.names.index('scenario')
    i.insert(i_scenario,maps['scenario_name'].keys())
    return pd.MultiIndex.from_product(i,names=mI.names)

def add_zeros(df,category):
    i = [x for x in df.columns if x!=0]
    df = df.set_index(i)
    
    mI = create_expanded_tech_MultiIndex(df.index,category)
    
    df = df.reindex(mI,fill_value = 0).reset_index()
    return df

def aggregate_techs(df, agg_list):
    id_list = [cols for cols in df.columns if cols not in ['alltec',0]]
    for agg_tech in agg_list:
        df = pd.concat([df,
                     df[df['alltec'].isin(maps['aggregated_techs'][agg_tech])]\
                     .groupby(id_list).sum()\
                     .assign(alltec=agg_tech)\
                     .reset_index()])
    return df

def add_index_cols(df,prefix,preprefix):
    ident = prefix+'|'+preprefix
    df['Updated code'] = df['alltec'].apply(lambda x: replace(x, prepend=ident, d=maps['technology']))
    df['Scenario'] = df['scenario'].map(maps['scenario_name'])
    df['Year'] = df['scenario'].apply(lambda x: 'Y_'+str(scenarios[to_scenarioID[x]]['clp']['--HORIZON']))
    df['Model'] = 'EMMA'
    df['Unit'] = maps['unit_per_category'][prefix]
    df['Region'] = df['r']
    return df

def rename_et_al(df,category):
    df = df.astype({0: 'float64'})
    df = add_zeros(df,category)
    df = aggregate_techs(df, maps['categories_agg_techs'][category])
    df = add_index_cols(df,prefix,category)
    return df

IAMC = {}

##### Installed_capacity_supply (to be reported in GW)

In [4]:
prefix = 'Installed capacity'

df = dh.get('o_capa').xs('GER', level='r', drop_level=False).stack().reset_index()
df = df[df['alltec']!='shed']

df_chp = dh.get('o_capachp').xs('GER', level='r', drop_level=False).stack().reset_index()
df_chp.columns = change_tec_lvl_name_to_alltec(df_chp.columns)
df_chp = df_chp[df_chp['alltec']!='shed']

elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

ntc_capa = dh.get('o_ntc_capa').groupby(['r']).sum().reset_index()\
                               .assign(alltec='ntc',allvin=1)\
                               .melt(id_vars=['alltec','allvin','r'],value_name=0)
ntc_capa = ntc_capa[ntc_capa['r']=='GER']
flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = pd.concat([flex,ntc_capa])
flex = rename_et_al(flex,'Flexibility')

hydrogen = df[df['alltec'].isin(maps['categories']['Hydrogen'])]
hydrogen = rename_et_al(hydrogen,'Hydrogen')

heat = rename_et_al(df_chp,'Heat')

df = pd.concat([elec,flex,heat,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['installed_capacity'] = df.copy()
IAMC['installed_capacity']

Year                                                                                     Y_2016  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Installed capacity|Electricity                     GW       NaN   
                                Installed capacity|Electricity|CCS                 GW       NaN   
                                Installed capacity|Electricity|Coal                GW       NaN   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW       NaN   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW       NaN   
...                                                                                         ...   
      Reference_year     GER    Installed capacity|Heat|Coal|Hard Coal|CCS         GW       0.0   
                                Installed capacity|Heat|Gases|Fossil|Natural gas   GW       0.0   
                                Installed capacity|Heat|Gases|Fossil|Natural ga... GW       0.0   
                                Installed capacity|Heat|Gases|Hydrogen             GW       0.0   
                                Installed capacity|Hydrogen|Electricity            GW       0.0   

Year                                                                                         Y_2030  \
Model Scenario           Region Updated code                                       Unit               
EMMA  Climate neutrality GER    Installed capacity|Electricity                     GW    150.522956   
                                Installed capacity|Electricity|CCS                 GW      0.000000   
                                Installed capacity|Electricity|Coal                GW     17.000000   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW      9.000000   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW      0.000000   
...                                                                                             ...   
      Reference_year     GER    Installed capacity|Heat|Coal|Hard Coal|CCS         GW           NaN   
                                Installed capacity|Heat|Gases|Fossil|Natural gas   GW           NaN   
                                Installed capacity|Heat|Gases|Fossil|Natural ga... GW           NaN   
                                Installed capacity|Heat|Gases|Hydrogen             GW           NaN   
                                Installed capacity|Hydrogen|Electricity            GW           NaN   

Year                                                                                     Y_2040  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Installed capacity|Electricity                     GW       NaN   
                                Installed capacity|Electricity|CCS                 GW       NaN   
                                Installed capacity|Electricity|Coal                GW       NaN   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW       NaN   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW       NaN   
...                                                                                         ...   
      Reference_year     GER    Installed capacity|Heat|Coal|Hard Coal|CCS         GW       NaN   
                                Installed capacity|Heat|Gases|Fossil|Natural gas   GW       NaN   
                                Installed capacity|Heat|Gases|Fossil|Natural ga... GW       NaN   
                                Installed capacity|Heat|Gases|Hydrogen             GW       NaN   
                                Installed capacity|Hydrogen|Electricity            GW       NaN   

Year                                                                                

##### yearly_generation_supply (to be reported in GWh)

In [5]:
prefix = 'Generation|Yearly'

df = dh.get('o_supply').xs('GER', level='r', drop_level=False).groupby(['tec_supply','allvin','r']).sum().stack().reset_index()
df.columns = change_tec_lvl_name_to_alltec(df.columns)
df = df[df['alltec']!='shed']
elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

flow = dh.get('o_flow').groupby(['r']).sum().mul(-1).reset_index()\
                               .assign(alltec='ntc',allvin=1)\
                               .melt(id_vars=['alltec','allvin','r'],value_name=0)
flow = flow[flow['r']=='GER']
flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = pd.concat([flex,flow])
flex = rename_et_al(flex,'Flexibility')

hydrogen = dh.get('o_demand').xs('GER', level='r', drop_level=False).xs('PtHydrogen', level='tec_demand', drop_level=False)
eff = dh.get('eff').loc['PtHydrogen',:]
hydrogen = pd.concat(
    [hydrogen[col].mul(eff[col]) for col in hydrogen.columns],
    axis=1
)
hydrogen.columns = eff.index
hydrogen = hydrogen.stack().reset_index()
hydrogen.columns = change_tec_lvl_name_to_alltec(hydrogen.columns)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['yearly_generation_supply'] = df.copy()
IAMC['yearly_generation_supply']

Year                                                                                     Y_2016  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Yearly|Electricity                      GWh      NaN   
                                Generation|Yearly|Electricity|CCS                  GWh      NaN   
                                Generation|Yearly|Electricity|Coal                 GWh      NaN   
                                Generation|Yearly|Electricity|Coal|Brown Coal|L... GWh      NaN   
                                Generation|Yearly|Electricity|Coal|Brown Coal|L... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Yearly|Flexibility|Electricity Storage  GWh      0.0   
                                Generation|Yearly|Flexibility|Electricity Stora... GWh      0.0   
                                Generation|Yearly|Flexibility|Electricity Stora... GWh      0.0   
                                Generation|Yearly|Flexibility|Interconnect Impo... GWh      0.0   
                                Generation|Yearly|Hydrogen|Electricity             GWh      0.0   

Year                                                                                            Y_2030  \
Model Scenario           Region Updated code                                       Unit                  
EMMA  Climate neutrality GER    Generation|Yearly|Electricity                      GWh   384227.294527   
                                Generation|Yearly|Electricity|CCS                  GWh        0.000000   
                                Generation|Yearly|Electricity|Coal                 GWh    94846.223672   
                                Generation|Yearly|Electricity|Coal|Brown Coal|L... GWh    53297.590721   
                                Generation|Yearly|Electricity|Coal|Brown Coal|L... GWh        0.000000   
...                                                                                                ...   
      Reference_year     GER    Generation|Yearly|Flexibility|Electricity Storage  GWh             NaN   
                                Generation|Yearly|Flexibility|Electricity Stora... GWh             NaN   
                                Generation|Yearly|Flexibility|Electricity Stora... GWh             NaN   
                                Generation|Yearly|Flexibility|Interconnect Impo... GWh             NaN   
                                Generation|Yearly|Hydrogen|Electricity             GWh             NaN   

Year                                                                                     Y_2040  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Yearly|Electricity                      GWh      NaN   
                                Generation|Yearly|Electricity|CCS                  GWh      NaN   
                                Generation|Yearly|Electricity|Coal                 GWh      NaN   
                                Generation|Yearly|Electricity|Coal|Brown Coal|L... GWh      NaN   
                                Generation|Yearly|Electricity|Coal|Brown Coal|L... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Yearly|Flexibility|Electricity Storage  GWh      NaN   
                                Generation|Yearly|Flexibility|Electricity Stora... GWh      NaN   
                                Generation|Yearly|Flexibility|Electricity Stora... GWh      NaN   
                                Generation|Yearly|Flexibility|Interconnect Impo... GWh      NaN   
                                Generation|Yearly|Hydrogen|Electricity             GWh      NaN   

Year                                         

##### summer_peak_hourly_generation (to be reported in GWh)

In [6]:
from datetime import datetime, timedelta

In [7]:
prefix = 'Generation|Summer peak'

exo_demand = dh.get('o_load').xs('GER', level='r')
endo_demand = dh.get('o_demand').xs('GER', level='r').groupby(['t']).sum().fillna(0)

summer_peak = exo_demand.add(endo_demand).loc[list(range(172*24, 264*24+1))].idxmax()


df = dh.get('o_supply').xs('GER', level='r', drop_level=False)
df = pd.concat(
    [df.xs(summer_peak[col], level='t')[col] for col in df.columns],
    axis=1,
)
df.columns = df.columns.rename('scenario')
df = df.stack().reset_index()
df.columns = change_tec_lvl_name_to_alltec(df.columns)
df = df[df['alltec']!='shed']

elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

flow = dh.get('o_flow').groupby(['t','r']).sum().xs('GER', level='r', drop_level=False).mul(-1)
flow = pd.concat(
    [flow.xs(summer_peak[col], level='t')[col] for col in flow.columns],
    axis=1,
)
flow.columns = flow.columns.rename('scenario')
flow = flow.stack().reset_index().assign(alltec='ntc',allvin=1)

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = pd.concat([flex,flow])
flex = rename_et_al(flex,'Flexibility')

hydrogen = dh.get('o_demand').xs('GER', level='r', drop_level=False).xs('PtHydrogen', level='tec_demand', drop_level=False)
eff = dh.get('eff').loc['PtHydrogen',:]
hydrogen = pd.concat(
    [hydrogen.xs(summer_peak[col], level='t')[col].mul(eff[col]) for col in hydrogen.columns],
    axis=1
)
hydrogen.columns = eff.index
hydrogen = hydrogen.stack().reset_index()
hydrogen.columns = change_tec_lvl_name_to_alltec(hydrogen.columns)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['summer_peak_hourly_generation'] = df.copy()
IAMC['summer_peak_hourly_generation']

Year                                                                                     Y_2016  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Summer peak|Electricity                 GWh      NaN   
                                Generation|Summer peak|Electricity|CCS             GWh      NaN   
                                Generation|Summer peak|Electricity|Coal            GWh      NaN   
                                Generation|Summer peak|Electricity|Coal|Brown C... GWh      NaN   
                                Generation|Summer peak|Electricity|Coal|Brown C... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Summer peak|Flexibility|Electricity ... GWh      0.0   
                                Generation|Summer peak|Flexibility|Electricity ... GWh      0.0   
                                Generation|Summer peak|Flexibility|Electricity ... GWh      0.0   
                                Generation|Summer peak|Flexibility|Interconnect... GWh      0.0   
                                Generation|Summer peak|Hydrogen|Electricity        GWh      0.0   

Year                                                                                        Y_2030  \
Model Scenario           Region Updated code                                       Unit              
EMMA  Climate neutrality GER    Generation|Summer peak|Electricity                 GWh   69.501584   
                                Generation|Summer peak|Electricity|CCS             GWh    0.000000   
                                Generation|Summer peak|Electricity|Coal            GWh    7.578899   
                                Generation|Summer peak|Electricity|Coal|Brown C... GWh    6.079877   
                                Generation|Summer peak|Electricity|Coal|Brown C... GWh    0.000000   
...                                                                                            ...   
      Reference_year     GER    Generation|Summer peak|Flexibility|Electricity ... GWh         NaN   
                                Generation|Summer peak|Flexibility|Electricity ... GWh         NaN   
                                Generation|Summer peak|Flexibility|Electricity ... GWh         NaN   
                                Generation|Summer peak|Flexibility|Interconnect... GWh         NaN   
                                Generation|Summer peak|Hydrogen|Electricity        GWh         NaN   

Year                                                                                     Y_2040  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Summer peak|Electricity                 GWh      NaN   
                                Generation|Summer peak|Electricity|CCS             GWh      NaN   
                                Generation|Summer peak|Electricity|Coal            GWh      NaN   
                                Generation|Summer peak|Electricity|Coal|Brown C... GWh      NaN   
                                Generation|Summer peak|Electricity|Coal|Brown C... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Summer peak|Flexibility|Electricity ... GWh      NaN   
                                Generation|Summer peak|Flexibility|Electricity ... GWh      NaN   
                                Generation|Summer peak|Flexibility|Electricity ... GWh      NaN   
                                Generation|Summer peak|Flexibility|Interconnect... GWh      NaN   
                                Generation|Summer peak|Hydrogen|Electricity        GWh      NaN   

Year                                                                                         Y_20

##### winter_peak_hourly_generation (to be reported in GWh)

In [8]:
prefix = 'Generation|Winter peak'

exo_demand = dh.get('o_load').xs('GER', level='r')
endo_demand = dh.get('o_demand').xs('GER', level='r').groupby(['t']).sum().fillna(0)

winter_peak = exo_demand.add(endo_demand).loc[list(range(1,79*24)) + list(range(355*24,8760+1))].idxmax()


df = dh.get('o_supply').xs('GER', level='r', drop_level=False)
df = pd.concat(
    [df.xs(winter_peak[col], level='t')[col] for col in df.columns],
    axis=1,
)
df.columns = df.columns.rename('scenario')
df = df.stack().reset_index()
df.columns = change_tec_lvl_name_to_alltec(df.columns)
df = df[df['alltec']!='shed']

elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

flow = dh.get('o_flow').groupby(['t','r']).sum().xs('GER', level='r', drop_level=False).mul(-1)
flow = pd.concat(
    [flow.xs(winter_peak[col], level='t')[col] for col in flow.columns],
    axis=1,
)
flow.columns = flow.columns.rename('scenario')
flow = flow.stack().reset_index().assign(alltec='ntc',allvin=1)

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = pd.concat([flex,flow])
flex = rename_et_al(flex,'Flexibility')

hydrogen = dh.get('o_demand').xs('GER', level='r', drop_level=False).xs('PtHydrogen', level='tec_demand', drop_level=False)
eff = dh.get('eff').loc['PtHydrogen',:]
hydrogen = pd.concat(
    [hydrogen.xs(winter_peak[col], level='t')[col].mul(eff[col]) \
     if winter_peak[col] in hydrogen[col].index.get_level_values('t').values \
     else pd.DataFrame({col:0},index=pd.MultiIndex.from_product([['PtHydrogen'],['new'],['GER']],names=['tec_demand','allvin','r']))\
     for col in hydrogen.columns],
    axis=1
)
hydrogen.columns = eff.index
hydrogen = hydrogen.stack().reset_index()
hydrogen.columns = change_tec_lvl_name_to_alltec(hydrogen.columns)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['winter_peak_hourly_generation'] = df.copy()
IAMC['winter_peak_hourly_generation']

Year                                                                                     Y_2016  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Winter peak|Electricity                 GWh      NaN   
                                Generation|Winter peak|Electricity|CCS             GWh      NaN   
                                Generation|Winter peak|Electricity|Coal            GWh      NaN   
                                Generation|Winter peak|Electricity|Coal|Brown C... GWh      NaN   
                                Generation|Winter peak|Electricity|Coal|Brown C... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Winter peak|Flexibility|Electricity ... GWh      0.0   
                                Generation|Winter peak|Flexibility|Electricity ... GWh      0.0   
                                Generation|Winter peak|Flexibility|Electricity ... GWh      0.0   
                                Generation|Winter peak|Flexibility|Interconnect... GWh      0.0   
                                Generation|Winter peak|Hydrogen|Electricity        GWh      0.0   

Year                                                                                        Y_2030  \
Model Scenario           Region Updated code                                       Unit              
EMMA  Climate neutrality GER    Generation|Winter peak|Electricity                 GWh   67.548833   
                                Generation|Winter peak|Electricity|CCS             GWh    0.000000   
                                Generation|Winter peak|Electricity|Coal            GWh    3.129320   
                                Generation|Winter peak|Electricity|Coal|Brown C... GWh    0.888890   
                                Generation|Winter peak|Electricity|Coal|Brown C... GWh    0.000000   
...                                                                                            ...   
      Reference_year     GER    Generation|Winter peak|Flexibility|Electricity ... GWh         NaN   
                                Generation|Winter peak|Flexibility|Electricity ... GWh         NaN   
                                Generation|Winter peak|Flexibility|Electricity ... GWh         NaN   
                                Generation|Winter peak|Flexibility|Interconnect... GWh         NaN   
                                Generation|Winter peak|Hydrogen|Electricity        GWh         NaN   

Year                                                                                     Y_2040  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Winter peak|Electricity                 GWh      NaN   
                                Generation|Winter peak|Electricity|CCS             GWh      NaN   
                                Generation|Winter peak|Electricity|Coal            GWh      NaN   
                                Generation|Winter peak|Electricity|Coal|Brown C... GWh      NaN   
                                Generation|Winter peak|Electricity|Coal|Brown C... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Winter peak|Flexibility|Electricity ... GWh      NaN   
                                Generation|Winter peak|Flexibility|Electricity ... GWh      NaN   
                                Generation|Winter peak|Flexibility|Electricity ... GWh      NaN   
                                Generation|Winter peak|Flexibility|Interconnect... GWh      NaN   
                                Generation|Winter peak|Hydrogen|Electricity        GWh      NaN   

Year                                                                                         Y_20

##### Percentile25_hourly_generation (to be reported in GWh)

In [9]:
exo_demand = dh.get('o_load').xs('GER', level='r')
endo_demand = dh.get('o_demand').xs('GER', level='r').groupby(['t']).sum().fillna(0)

demand = exo_demand.add(endo_demand)

descending_demand_index = {
    scen:demand[scen].sort_values(ascending=False).index.values
    for scen in demand.columns
}
descending_demand_index = pd.DataFrame(descending_demand_index)

In [10]:
perc_25 = descending_demand_index.iloc[:round(len(descending_demand_index)*0.25)]
nr_of_hours = len(perc_25)

prefix = 'Generation|Percentile 25'

df = dh.get('o_supply').xs('GER', level='r', drop_level=False)
df = pd.concat(
    [df[scen].loc[(perc_25[scen]),:]
     for scen in df.columns], axis=1
)
df.columns = df.columns.rename('scenario')
df = df.stack().reset_index()
df.columns = change_tec_lvl_name_to_alltec(df.columns)
df = df[df['alltec']!='shed']

elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

flow = dh.get('o_flow').groupby(['t','r']).sum().xs('GER', level='r', drop_level=False).mul(-1)
flow = pd.concat(
    [flow[scen].loc[(perc_25[scen]),:]
     for scen in flow.columns], axis=1
)
flow.columns = flow.columns.rename('scenario')
flow = flow.stack().reset_index().assign(alltec='ntc',allvin=1)

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = pd.concat([flex,flow])
flex = rename_et_al(flex,'Flexibility')

hydrogen = dh.get('o_demand').xs('GER', level='r', drop_level=False).xs('PtHydrogen', level='tec_demand', drop_level=False)
eff = dh.get('eff').loc['PtHydrogen',:]
hydrogen = pd.concat(
    [hydrogen[col].loc[(perc_25[col]),:].mul(eff[col]) for col in hydrogen.columns],
    axis=1
)
hydrogen.columns = eff.index
hydrogen = hydrogen.stack().reset_index()
hydrogen.columns = change_tec_lvl_name_to_alltec(hydrogen.columns)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year').div(nr_of_hours) # .to_csv('test.csv')

IAMC['Percentile25_hourly_generation'] = df.copy()
IAMC['Percentile25_hourly_generation']

Year                                                                                     Y_2016  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Percentile 25|Electricity               GWh      NaN   
                                Generation|Percentile 25|Electricity|CCS           GWh      NaN   
                                Generation|Percentile 25|Electricity|Coal          GWh      NaN   
                                Generation|Percentile 25|Electricity|Coal|Brown... GWh      NaN   
                                Generation|Percentile 25|Electricity|Coal|Brown... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Percentile 25|Flexibility|Electricit... GWh      0.0   
                                Generation|Percentile 25|Flexibility|Electricit... GWh      0.0   
                                Generation|Percentile 25|Flexibility|Electricit... GWh      0.0   
                                Generation|Percentile 25|Flexibility|Interconne... GWh      0.0   
                                Generation|Percentile 25|Hydrogen|Electricity      GWh      0.0   

Year                                                                                        Y_2030  \
Model Scenario           Region Updated code                                       Unit              
EMMA  Climate neutrality GER    Generation|Percentile 25|Electricity               GWh   51.526177   
                                Generation|Percentile 25|Electricity|CCS           GWh    0.000000   
                                Generation|Percentile 25|Electricity|Coal          GWh   10.171821   
                                Generation|Percentile 25|Electricity|Coal|Brown... GWh    5.735370   
                                Generation|Percentile 25|Electricity|Coal|Brown... GWh    0.000000   
...                                                                                            ...   
      Reference_year     GER    Generation|Percentile 25|Flexibility|Electricit... GWh         NaN   
                                Generation|Percentile 25|Flexibility|Electricit... GWh         NaN   
                                Generation|Percentile 25|Flexibility|Electricit... GWh         NaN   
                                Generation|Percentile 25|Flexibility|Interconne... GWh         NaN   
                                Generation|Percentile 25|Hydrogen|Electricity      GWh         NaN   

Year                                                                                     Y_2040  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Percentile 25|Electricity               GWh      NaN   
                                Generation|Percentile 25|Electricity|CCS           GWh      NaN   
                                Generation|Percentile 25|Electricity|Coal          GWh      NaN   
                                Generation|Percentile 25|Electricity|Coal|Brown... GWh      NaN   
                                Generation|Percentile 25|Electricity|Coal|Brown... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Percentile 25|Flexibility|Electricit... GWh      NaN   
                                Generation|Percentile 25|Flexibility|Electricit... GWh      NaN   
                                Generation|Percentile 25|Flexibility|Electricit... GWh      NaN   
                                Generation|Percentile 25|Flexibility|Interconne... GWh      NaN   
                                Generation|Percentile 25|Hydrogen|Electricity      GWh      NaN   

Year                                                                                         Y_20

##### Percentile50_hourly_generation (to be reported in GWh)

In [11]:
perc_50 = descending_demand_index.iloc[:round(len(descending_demand_index)*0.5)]
nr_of_hours = len(perc_50)

prefix = 'Generation|Percentile 50'

df = dh.get('o_supply').xs('GER', level='r', drop_level=False)
df = pd.concat(
    [df[scen].loc[(perc_50[scen]),:]
     for scen in df.columns], axis=1
)
df.columns = df.columns.rename('scenario')
df = df.stack().reset_index()
df.columns = change_tec_lvl_name_to_alltec(df.columns)
df = df[df['alltec']!='shed']

elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

flow = dh.get('o_flow').groupby(['t','r']).sum().xs('GER', level='r', drop_level=False).mul(-1)
flow = pd.concat(
    [flow[scen].loc[(perc_50[scen]),:]
     for scen in flow.columns], axis=1
)
flow.columns = flow.columns.rename('scenario')
flow = flow.stack().reset_index().assign(alltec='ntc',allvin=1)

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = pd.concat([flex,flow])
flex = rename_et_al(flex,'Flexibility')

hydrogen = dh.get('o_demand').xs('GER', level='r', drop_level=False).xs('PtHydrogen', level='tec_demand', drop_level=False)
eff = dh.get('eff').loc['PtHydrogen',:]
hydrogen = pd.concat(
    [hydrogen[col].loc[(perc_50[col]),:].mul(eff[col]) for col in hydrogen.columns],
    axis=1
)
hydrogen.columns = eff.index
hydrogen = hydrogen.stack().reset_index()
hydrogen.columns = change_tec_lvl_name_to_alltec(hydrogen.columns)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year').div(nr_of_hours) # .to_csv('test.csv')

IAMC['Percentile50_hourly_generation'] = df.copy()
IAMC['Percentile50_hourly_generation']

Year                                                                                     Y_2016  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Percentile 50|Electricity               GWh      NaN   
                                Generation|Percentile 50|Electricity|CCS           GWh      NaN   
                                Generation|Percentile 50|Electricity|Coal          GWh      NaN   
                                Generation|Percentile 50|Electricity|Coal|Brown... GWh      NaN   
                                Generation|Percentile 50|Electricity|Coal|Brown... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Percentile 50|Flexibility|Electricit... GWh      0.0   
                                Generation|Percentile 50|Flexibility|Electricit... GWh      0.0   
                                Generation|Percentile 50|Flexibility|Electricit... GWh      0.0   
                                Generation|Percentile 50|Flexibility|Interconne... GWh      0.0   
                                Generation|Percentile 50|Hydrogen|Electricity      GWh      0.0   

Year                                                                                        Y_2030  \
Model Scenario           Region Updated code                                       Unit              
EMMA  Climate neutrality GER    Generation|Percentile 50|Electricity               GWh   48.401321   
                                Generation|Percentile 50|Electricity|CCS           GWh    0.000000   
                                Generation|Percentile 50|Electricity|Coal          GWh   10.098457   
                                Generation|Percentile 50|Electricity|Coal|Brown... GWh    5.709912   
                                Generation|Percentile 50|Electricity|Coal|Brown... GWh    0.000000   
...                                                                                            ...   
      Reference_year     GER    Generation|Percentile 50|Flexibility|Electricit... GWh         NaN   
                                Generation|Percentile 50|Flexibility|Electricit... GWh         NaN   
                                Generation|Percentile 50|Flexibility|Electricit... GWh         NaN   
                                Generation|Percentile 50|Flexibility|Interconne... GWh         NaN   
                                Generation|Percentile 50|Hydrogen|Electricity      GWh         NaN   

Year                                                                                     Y_2040  \
Model Scenario           Region Updated code                                       Unit           
EMMA  Climate neutrality GER    Generation|Percentile 50|Electricity               GWh      NaN   
                                Generation|Percentile 50|Electricity|CCS           GWh      NaN   
                                Generation|Percentile 50|Electricity|Coal          GWh      NaN   
                                Generation|Percentile 50|Electricity|Coal|Brown... GWh      NaN   
                                Generation|Percentile 50|Electricity|Coal|Brown... GWh      NaN   
...                                                                                         ...   
      Reference_year     GER    Generation|Percentile 50|Flexibility|Electricit... GWh      NaN   
                                Generation|Percentile 50|Flexibility|Electricit... GWh      NaN   
                                Generation|Percentile 50|Flexibility|Electricit... GWh      NaN   
                                Generation|Percentile 50|Flexibility|Interconne... GWh      NaN   
                                Generation|Percentile 50|Hydrogen|Electricity      GWh      NaN   

Year                                                                                         Y_20

##### fuel_consumption_supply (to be reported in TJ/year)

In [12]:
prefix = 'Fuel consumption'

eff = dh.get('efficiency')

df = dh.get('o_supply').xs('GER', level='r', drop_level=False).groupby(['tec_supply','allvin','r']).sum()
df.index.names = change_tec_lvl_name_to_alltec(df.index.names)
df = df.div(eff).mul(3.6) # From GWh to TJ
df = df.stack().reset_index()
df = df[df['alltec']!='shed']
elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
flex = rename_et_al(flex,'Flexibility')

hydrogen = dh.get('o_demand').xs('GER', level='r', drop_level=False).xs('PtHydrogen', level='tec_demand', drop_level=False)
hydrogen = hydrogen.stack().reset_index()
hydrogen.columns = change_tec_lvl_name_to_alltec(hydrogen.columns)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['fuel_consumption_supply'] = df.copy()
IAMC['fuel_consumption_supply']

Year                                                                                        Y_2016  \
Model Scenario           Region Updated code                                       Unit              
EMMA  Climate neutrality GER    Fuel consumption|Electricity                       TJ/year     NaN   
                                Fuel consumption|Electricity|CCS                   TJ/year     NaN   
                                Fuel consumption|Electricity|Coal                  TJ/year     NaN   
                                Fuel consumption|Electricity|Coal|Brown Coal|Li... TJ/year     NaN   
                                Fuel consumption|Electricity|Coal|Brown Coal|Li... TJ/year     NaN   
...                                                                                            ...   
      Reference_year     GER    Fuel consumption|Flexibility|Electricity Storage   TJ/year     0.0   
                                Fuel consumption|Flexibility|Electricity Storag... TJ/year     0.0   
                                Fuel consumption|Flexibility|Electricity Storag... TJ/year     0.0   
                                Fuel consumption|Flexibility|Interconnect Impor... TJ/year     0.0   
                                Fuel consumption|Hydrogen|Electricity              TJ/year     0.0   

Year                                                                                              Y_2030  \
Model Scenario           Region Updated code                                       Unit                    
EMMA  Climate neutrality GER    Fuel consumption|Electricity                       TJ/year  1.996068e+06   
                                Fuel consumption|Electricity|CCS                   TJ/year  0.000000e+00   
                                Fuel consumption|Electricity|Coal                  TJ/year  8.226041e+05   
                                Fuel consumption|Electricity|Coal|Brown Coal|Li... TJ/year  4.802114e+05   
                                Fuel consumption|Electricity|Coal|Brown Coal|Li... TJ/year  0.000000e+00   
...                                                                                                  ...   
      Reference_year     GER    Fuel consumption|Flexibility|Electricity Storage   TJ/year           NaN   
                                Fuel consumption|Flexibility|Electricity Storag... TJ/year           NaN   
                                Fuel consumption|Flexibility|Electricity Storag... TJ/year           NaN   
                                Fuel consumption|Flexibility|Interconnect Impor... TJ/year           NaN   
                                Fuel consumption|Hydrogen|Electricity              TJ/year           NaN   

Year                                                                                        Y_2040  \
Model Scenario           Region Updated code                                       Unit              
EMMA  Climate neutrality GER    Fuel consumption|Electricity                       TJ/year     NaN   
                                Fuel consumption|Electricity|CCS                   TJ/year     NaN   
                                Fuel consumption|Electricity|Coal                  TJ/year     NaN   
                                Fuel consumption|Electricity|Coal|Brown Coal|Li... TJ/year     NaN   
                                Fuel consumption|Electricity|Coal|Brown Coal|Li... TJ/year     NaN   
...                                                                                            ...   
      Reference_year     GER    Fuel consumption|Flexibility|Electricity Storage   TJ/year     NaN   
                                Fuel consumption|Flexibility|Electricity Storag... TJ/year     NaN   
                                Fuel consumption|Flexibility|Electricity Storag... TJ/year     NaN   
                                Fuel consumption|Flexibility|Interconnect Impor... TJ/year     NaN   
                                Fuel consump

##### emission_supply (to be reported in Mt/year)

In [13]:
prefix = 'Emissions|Kyoto Gases|Fossil|CO2'

df = dh.get('o_emissions').xs('GER', level='r',drop_level=False).stack().reset_index()
df.columns = change_tec_lvl_name_to_alltec(df.columns)
df = df[df['alltec']!='shed'].assign(allvin = 1)

elec = df[df['alltec'].isin(maps['categories']['Electricity'])]
elec = rename_et_al(elec,'Electricity')

hydrogen = pd.DataFrame({'alltec':'PtHydrogen','r':'GER','allvin':1,'scenario':elec['scenario'].unique(),0:0})
hydrogen = rename_et_al(hydrogen,'Hydrogen') 

df = pd.concat([elec,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['emission_supply'] = df.copy()
IAMC['emission_supply']

Year                                                                                            Y_2016  \
Model Scenario           Region Updated code                                       Unit                  
EMMA  Climate neutrality GER    Emissions|Kyoto Gases|Fossil|CO2|Electricity       Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|CCS   Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Coal  Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Co... Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Co... Mt CO2/year     NaN   
...                                                                                                ...   
      Reference_year     GER    Emissions|Kyoto Gases|Fossil|CO2|Electricity|So... Mt CO2/year     0.0   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wind  Mt CO2/year     0.0   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wi... Mt CO2/year     0.0   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wi... Mt CO2/year     0.0   
                                Emissions|Kyoto Gases|Fossil|CO2|Hydrogen|Elect... Mt CO2/year     0.0   

Year                                                                                                Y_2030  \
Model Scenario           Region Updated code                                       Unit                      
EMMA  Climate neutrality GER    Emissions|Kyoto Gases|Fossil|CO2|Electricity       Mt CO2/year  103.653260   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|CCS   Mt CO2/year    0.000000   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Coal  Mt CO2/year   86.321966   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Co... Mt CO2/year   53.720540   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Co... Mt CO2/year    0.000000   
...                                                                                                    ...   
      Reference_year     GER    Emissions|Kyoto Gases|Fossil|CO2|Electricity|So... Mt CO2/year         NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wind  Mt CO2/year         NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wi... Mt CO2/year         NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wi... Mt CO2/year         NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Hydrogen|Elect... Mt CO2/year         NaN   

Year                                                                                            Y_2040  \
Model Scenario           Region Updated code                                       Unit                  
EMMA  Climate neutrality GER    Emissions|Kyoto Gases|Fossil|CO2|Electricity       Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|CCS   Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Coal  Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Co... Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Co... Mt CO2/year     NaN   
...                                                                                                ...   
      Reference_year     GER    Emissions|Kyoto Gases|Fossil|CO2|Electricity|So... Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wind  Mt CO2/year     NaN   
                                Emissions|Kyoto Gases|Fossil|CO2|Electricity|Wi... Mt CO2/year     NaN   
                        

##### efficiency_supply

In [14]:
prefix = 'Efficiency'

df = dh.get('eff').stack().reset_index()
df = df[df['alltec']!='shed']
elec = df[df['alltec'].isin(maps['categories']['Electricity'])].assign(r='GER')
elec = add_index_cols(elec,prefix,'Electricity')

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
ntc_eff = pd.DataFrame({'alltec': 'ntc', 'scenario': flex['scenario'].unique(),0:1})
flex = pd.concat([flex,ntc_eff])
flex = flex.assign(r='GER')
flex = add_index_cols(flex,prefix,'Flexibility')

hydrogen = df[df['alltec'].isin(maps['categories']['Hydrogen'])].assign(r='GER')
hydrogen = add_index_cols(hydrogen,prefix,'Hydrogen')

df = pd.concat([elec,flex,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['efficiency_supply'] = df.copy()
IAMC['efficiency_supply']

Year                                                                                              Y_2030  \
Model Scenario           Region Updated code                                       Unit                    
EMMA  Climate neutrality GER    Efficiency|Electricity|Coal|Brown Coal|Lignite     dimensionless    0.42   
                                Efficiency|Electricity|Coal|Brown Coal|Lignite|CCS dimensionless    0.33   
                                Efficiency|Electricity|Coal|Hard Coal              dimensionless    0.46   
                                Efficiency|Electricity|Coal|Hard Coal|CCS          dimensionless    0.38   
                                Efficiency|Electricity|Gases|Fossil|Natural gas    dimensionless    1.00   
                                Efficiency|Electricity|Gases|Fossil|Natural gas... dimensionless    0.46   
                                Efficiency|Electricity|Gases|Hydrogen              dimensionless    1.00   
                                Efficiency|Electricity|Hydro|dam                   dimensionless    1.00   
                                Efficiency|Electricity|Hydro|river                 dimensionless    1.00   
                                Efficiency|Electricity|Nuclear                     dimensionless    0.38   
                                Efficiency|Electricity|Solar                       dimensionless    1.00   
                                Efficiency|Electricity|Solar bio and waste         dimensionless    1.00   
                                Efficiency|Electricity|Wind|Offshore               dimensionless    1.00   
                                Efficiency|Electricity|Wind|Onshore                dimensionless    1.00   
                                Efficiency|Flexibility|Electricity Storage|Medi... dimensionless    0.70   
                                Efficiency|Flexibility|Electricity Storage|Shor... dimensionless    0.92   
                                Efficiency|Flexibility|Interconnect Importing C... dimensionless    1.00   
                                Efficiency|Hydrogen|Electricity                    dimensionless    0.75   
      Current trends     GER    Efficiency|Electricity|Coal|Brown Coal|Lignite     dimensionless    0.42   
                                Efficiency|Electricity|Coal|Brown Coal|Lignite|CCS dimensionless    0.33   
                                Efficiency|Electricity|Coal|Hard Coal              dimensionless    0.46   
                                Efficiency|Electricity|Coal|Hard Coal|CCS          dimensionless    0.38   
                                Efficiency|Electricity|Gases|Fossil|Natural gas    dimensionless    1.00   
                                Efficiency|Electricity|Gases|Fossil|Natural gas... dimensionless    0.46   
                                Efficiency|Electricity|Gases|Hydrogen              dimensionless    1.00   
                                Efficiency|Electricity|Hydro|dam                   dimensionless    1.00   
                                Efficiency|Electricity|Hydro|river                 dimensionless    1.00   
                                Efficiency|Electricity|Nuclear                     dimensionless    0.38   
                                Efficiency|Electricity|Solar                       dimensionless    1.00   
                                Efficiency|Electricity|Solar bio and waste         dimensionless    1.00   
                                Efficiency|Electricity|Wind|Offshore               dimensionless    1.00   
                                Efficiency|Electricity|Wind|Onshore                dimensionless    1.00   
                                Efficiency|Flexibility|Electricity Storage|Medi... dimensionless    0.70   
                                Efficiency|Flexibility|Electricity Storage|Shor... dimensionless    0.92   
                                Efficiency|Flexibility|Interconnect Importing C... dimensionless    1.00   
    

##### investment_supply

In [15]:
prefix = 'Investments'

cost = dh.get('i_cost').xs('invest', level='par_cost').stack().reset_index()
cost.columns = ['alltec','Year','scenario',0]
cost = cost.set_index(['alltec','scenario','Year'])

inve = dh.get('o_inve').fillna(0).xs('GER', level='r').stack().reset_index()
inve['Year'] = inve['scenario'].apply(lambda x: str(scenarios[to_scenarioID[x]]['clp']['--HORIZON']))
inve.columns = ['alltec','allvin','scenario',0,'Year']
inve = inve.groupby(['alltec','scenario','Year']).sum()

df = inve.mul(cost).div(1000).reset_index().dropna()
df = df[df['alltec']!='shed']

elec = df[df['alltec'].isin(maps['categories']['Electricity'])].assign(r='GER',allvin=1)
elec = rename_et_al(elec,'Electricity')

flex = df[df['alltec'].isin(maps['categories']['Flexibility'])]
ntc_mI = pd.MultiIndex.from_product([['ntc'],flex['scenario'].unique(),flex['Year'].unique()],names = ['alltec','scenario','Year'])
ntc_inve = pd.DataFrame({0:0},index = ntc_mI).reset_index()
flex = pd.concat([flex,ntc_inve])
flex = flex.assign(r='GER',allvin=1)
flex = rename_et_al(flex,'Flexibility')

hydrogen = df[df['alltec'].isin(maps['categories']['Hydrogen'])].assign(r='GER',allvin=1)
hydrogen = rename_et_al(hydrogen,'Hydrogen')

heat_inve = dh.get('o_invechp').fillna(0).xs('GER', level='r').stack().reset_index()
heat_inve['Year'] = heat_inve['scenario'].apply(lambda x: str(scenarios[to_scenarioID[x]]['clp']['--HORIZON']))
heat_inve.columns = change_tec_lvl_name_to_alltec(heat_inve.columns)
heat_inve = heat_inve.astype({0: 'Float64'})
heat_inve = heat_inve.groupby(['alltec','scenario','Year']).sum()

heat = heat_inve.mul(cost).div(1000).reset_index().dropna()
heat = heat[heat['alltec']!='shed'].assign(r='GER',allvin=1)
heat = rename_et_al(heat,'Heat')

df = pd.concat([elec,flex,heat,hydrogen])
df = df.astype({0: 'float64'})
df = df.groupby(index, dropna=False).sum()[0].unstack('Year') # .to_csv('test.csv')

IAMC['investment_supply'] = df.copy()
IAMC['investment_supply']

Year                                                                                                                       Y_2016  \
Model Scenario           Region Updated code                                       Unit                                             
EMMA  Climate neutrality GER    Investments|Electricity                            Time frame,  billion Euros (PPP, 2015)     NaN   
                                Investments|Electricity|CCS                        Time frame,  billion Euros (PPP, 2015)     NaN   
                                Investments|Electricity|Coal                       Time frame,  billion Euros (PPP, 2015)     NaN   
                                Investments|Electricity|Coal|Brown Coal|Lignite    Time frame,  billion Euros (PPP, 2015)     NaN   
                                Investments|Electricity|Coal|Brown Coal|Lignite... Time frame,  billion Euros (PPP, 2015)     NaN   
...                                                                                                                           ...   
      Reference_year     GER    Investments|Heat|Coal|Hard Coal|CCS                Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Heat|Gases|Fossil|Natural gas          Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Heat|Gases|Fossil|Natural gas|CCS      Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Heat|Gases|Hydrogen                    Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Hydrogen|Electricity                   Time frame,  billion Euros (PPP, 2015)     0.0   

Year                                                                                                                        Y_2030  \
Model Scenario           Region Updated code                                       Unit                                              
EMMA  Climate neutrality GER    Investments|Electricity                            Time frame,  billion Euros (PPP, 2015)  1.59159   
                                Investments|Electricity|CCS                        Time frame,  billion Euros (PPP, 2015)  0.00000   
                                Investments|Electricity|Coal                       Time frame,  billion Euros (PPP, 2015)  0.00000   
                                Investments|Electricity|Coal|Brown Coal|Lignite    Time frame,  billion Euros (PPP, 2015)  0.00000   
                                Investments|Electricity|Coal|Brown Coal|Lignite... Time frame,  billion Euros (PPP, 2015)  0.00000   
...                                                                                                                            ...   
      Reference_year     GER    Investments|Heat|Coal|Hard Coal|CCS                Time frame,  billion Euros (PPP, 2015)      NaN   
                                Investments|Heat|Gases|Fossil|Natural gas          Time frame,  billion Euros (PPP, 2015)      NaN   
                                Investments|Heat|Gases|Fossil|Natural gas|CCS      Time frame,  billion Euros (PPP, 2015)      NaN   
                                Investments|Heat|Gases|Hydrogen                    Time frame,  billion Euros (PPP, 2015)      NaN   
                                Investments|Hydrogen|Electricity                   Time frame,  billion Euros (PPP, 2015)      NaN   

Year                                                                                                                       Y_2040  \
Model Scenario           Region Updated code                                       Unit                                             
EMMA  Climate neutrality GER    Investments|Electricity                            Time frame,  billion Euros (PPP, 2015)     NaN   
                                Investments|Electricity|CCS                        Time frame,  billion Euros (PPP, 2015)     Na

In [21]:
IAMC['long'] = pd.concat(IAMC.values()).copy()

Year                                                                                                                       Y_2016  \
Model Scenario           Region Updated code                                       Unit                                             
EMMA  Climate neutrality GER    Installed capacity|Electricity                     GW                                         NaN   
                                Installed capacity|Electricity|CCS                 GW                                         NaN   
                                Installed capacity|Electricity|Coal                GW                                         NaN   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW                                         NaN   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW                                         NaN   
...                                                                                                                           ...   
      Reference_year     GER    Investments|Heat|Coal|Hard Coal|CCS                Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Heat|Gases|Fossil|Natural gas          Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Heat|Gases|Fossil|Natural gas|CCS      Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Heat|Gases|Hydrogen                    Time frame,  billion Euros (PPP, 2015)     0.0   
                                Investments|Hydrogen|Electricity                   Time frame,  billion Euros (PPP, 2015)     0.0   

Year                                                                                                                           Y_2030  \
Model Scenario           Region Updated code                                       Unit                                                 
EMMA  Climate neutrality GER    Installed capacity|Electricity                     GW                                      150.522956   
                                Installed capacity|Electricity|CCS                 GW                                        0.000000   
                                Installed capacity|Electricity|Coal                GW                                       17.000000   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW                                        9.000000   
                                Installed capacity|Electricity|Coal|Brown Coal|... GW                                        0.000000   
...                                                                                                                               ...   
      Reference_year     GER    Investments|Heat|Coal|Hard Coal|CCS                Time frame,  billion Euros (PPP, 2015)         NaN   
                                Investments|Heat|Gases|Fossil|Natural gas          Time frame,  billion Euros (PPP, 2015)         NaN   
                                Investments|Heat|Gases|Fossil|Natural gas|CCS      Time frame,  billion Euros (PPP, 2015)         NaN   
                                Investments|Heat|Gases|Hydrogen                    Time frame,  billion Euros (PPP, 2015)         NaN   
                                Investments|Hydrogen|Electricity                   Time frame,  billion Euros (PPP, 2015)         NaN   

Year                                                                                                                       Y_2040  \
Model Scenario           Region Updated code                                       Unit                                             
EMMA  Climate neutrality GER    Installed capacity|Electricity                     GW                                         NaN   
                                Installed capacity|Electricity|CCS                 GW    

In [18]:
IAMC['long'].to_excel("IAMC.xlsx",sheet_name='all_codes',merge_cells = False)